In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate
import ipywidgets as widgets
from IPython.display import display
# Load datasets
movies = pd.read_csv('/content/movies.csv')
ratings = pd.read_csv('/content/ratings.csv')

# Handles missing values
movies['genres'] = movies['genres'].fillna('')

# Calculate average ratings and number of reviews per movie
movie_ratings = ratings.groupby('movieId')['rating'].agg(['mean', 'count']).reset_index()
movie_ratings.columns = ['movieId', 'average_rating', 'num_reviews']

# Merge movie ratings with movies dataset
movies = movies.merge(movie_ratings, on='movieId', how='left')

# --- Exploratory Data Analysis (EDA) ---
# Total unique users and movies
unique_users = ratings['userId'].nunique()
unique_movies = ratings['movieId'].nunique()

print(f"Total Unique Users: {unique_users}")
print(f"Total Unique Movies: {unique_movies}")

# Distribution of ratings
rating_distribution = ratings['rating'].value_counts().sort_index()
print("\nRating Distribution:")
print(tabulate(rating_distribution.reset_index(), headers=['Rating', 'Count'], tablefmt='fancy_grid'))

# Average rating and total movies at genre level
def analyze_genre_distribution(movies):
    genre_analysis = []
    for index, row in movies.iterrows():
        genres = row['genres'].split('|')
        for genre in genres:
            genre_analysis.append({'genre': genre, 'movieId': row['movieId'], 'average_rating': row.get('average_rating', np.nan)})

    genre_df = pd.DataFrame(genre_analysis)
    genre_stats = genre_df.groupby('genre').agg(
        num_movies=('movieId', 'nunique'),
        avg_rating=('average_rating', 'mean')
    ).reset_index()
    return genre_stats

genre_stats = analyze_genre_distribution(movies)
print("\nGenre-Level Analysis:")
print(tabulate(genre_stats, headers=['Genre', 'Number of Movies', 'Average Rating'], tablefmt='fancy_grid'))

# Unique genres considered
unique_genres = set('|'.join(movies['genres']).split('|'))
print("\nUnique Genres:")
print(", ".join(unique_genres))

# Top 10 most reviewed movies
most_reviewed_movies = movies.sort_values(by='num_reviews', ascending=False).head(10)
print("\nTop 10 Most Reviewed Movies:")
print(tabulate(most_reviewed_movies[['title', 'num_reviews']], headers=['Title', 'Number of Reviews'], tablefmt='fancy_grid'))

# Top 10 highest-rated movies
highest_rated_movies = movies[movies['num_reviews'] >= 50].sort_values(by='average_rating', ascending=False).head(10)
print("\nTop 10 Highest-Rated Movies (with at least 50 reviews):")
print(tabulate(highest_rated_movies[['title', 'average_rating', 'num_reviews']], headers=['Title', 'Average Rating', 'Number of Reviews'], tablefmt='fancy_grid'))






Total Unique Users: 668
Total Unique Movies: 10325

Rating Distribution:
╒════╤══════════╤═════════╕
│    │   Rating │   Count │
╞════╪══════════╪═════════╡
│  0 │      0.5 │    1198 │
├────┼──────────┼─────────┤
│  1 │      1   │    3258 │
├────┼──────────┼─────────┤
│  2 │      1.5 │    1567 │
├────┼──────────┼─────────┤
│  3 │      2   │    7943 │
├────┼──────────┼─────────┤
│  4 │      2.5 │    5484 │
├────┼──────────┼─────────┤
│  5 │      3   │   21729 │
├────┼──────────┼─────────┤
│  6 │      3.5 │   12237 │
├────┼──────────┼─────────┤
│  7 │      4   │   28880 │
├────┼──────────┼─────────┤
│  8 │      4.5 │    8187 │
├────┼──────────┼─────────┤
│  9 │      5   │   14856 │
╘════╧══════════╧═════════╛

Genre-Level Analysis:
╒════╤════════════════════╤════════════════════╤══════════════════╕
│    │ Genre              │   Number of Movies │   Average Rating │
╞════╪════════════════════╪════════════════════╪══════════════════╡
│  0 │ (no genres listed) │                  7 │        

In [ ]:


# Popularity-based Recommender
def popularity_based_recommender(movies, ratings, genre, min_reviews, top_n):
    genre_movies = movies[movies['genres'].str.contains(genre, case=False)]
    genre_movie_ratings = genre_movies[genre_movies['num_reviews'] >= min_reviews]
    genre_movie_ratings = genre_movie_ratings.sort_values(by='average_rating', ascending=False).head(top_n)
    return genre_movie_ratings[['title', 'average_rating', 'num_reviews']]

# Content-based Recommender
def content_based_recommender(movies, movie_title, top_n):
    if movie_title not in movies['title'].values:#Verifies if the movie exists in the dataset.
        return f"Movie '{movie_title}' not found in the dataset."

    movie_genres = movies[movies['title'] == movie_title]['genres'].values[0]
    similar_movies = movies[movies['genres'] == movie_genres]
    similar_movies = similar_movies[similar_movies['title'] != movie_title]

    if similar_movies.empty:
        return f"No similar movies found for '{movie_title}'."

    return similar_movies[['title']].head(top_n)

# Collaborative-based Recommender
def collaborative_recommender(ratings, movies, target_user_id, top_k, top_n):
    user_item_matrix = ratings.pivot(index='userId', columns='movieId', values='rating').fillna(0)
    user_similarity = cosine_similarity(user_item_matrix)
    user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)

    if target_user_id not in user_similarity_df.index:
        return f"User ID {target_user_id} not found in the dataset."

    similar_users = user_similarity_df[target_user_id].sort_values(ascending=False).head(top_k).index
    similar_user_ratings = ratings[ratings['userId'].isin(similar_users)]
    #Aggregates ratings from these similar users to calculate the average rating for each movie.
    movie_recommendations = similar_user_ratings.groupby('movieId').agg(
        avg_rating=('rating', 'mean')
    ).reset_index()
    movie_recommendations = movie_recommendations.sort_values(by='avg_rating', ascending=False).head(top_n)
    #Sorts the movies by average rating and selects the top n movies.
    recommended_movies = pd.merge(movie_recommendations, movies[['movieId', 'title']], on='movieId')
    return recommended_movies[['title', 'avg_rating']]


# Example Usage
# Popularity-based Recommender
print("Popularity-based Recommendations:")
popular_movies = popularity_based_recommender(movies, ratings, genre='Comedy', min_reviews=100, top_n=5)
print(tabulate(popular_movies, headers=['Title', 'Average Rating', 'Num Reviews'], tablefmt='fancy_grid'))

# Content-based Recommender
print("\nContent-based Recommendations:")
recommended_movies = content_based_recommender(movies, movie_title='Toy Story (1995)', top_n=5)
if isinstance(recommended_movies, pd.DataFrame):
    print(tabulate(recommended_movies, headers=['Title'], tablefmt='fancy_grid'))
else:
    print(recommended_movies)

# Collaborative-based Recommender
print("\nCollaborative Recommendations:")
collaborative_movies = collaborative_recommender(ratings, movies, target_user_id=1, top_k=100, top_n=5)
if isinstance(collaborative_movies, pd.DataFrame):
    print(tabulate(collaborative_movies, headers=['Title', 'Average Rating'], tablefmt='fancy_grid'))
else:
    print(collaborative_movies)



Popularity-based Recommendations:
╒═════╤════════════════════════════════════════╤══════════════════╤═══════════════╕
│     │ Title                                  │   Average Rating │   Num Reviews │
╞═════╪════════════════════════════════════════╪══════════════════╪═══════════════╡
│ 922 │ Monty Python and the Holy Grail (1975) │          4.30195 │           154 │
├─────┼────────────────────────────────────────┼──────────────────┼───────────────┤
│ 537 │ Fargo (1996)                           │          4.27114 │           201 │
├─────┼────────────────────────────────────────┼──────────────────┼───────────────┤
│ 959 │ Princess Bride, The (1987)             │          4.16374 │           171 │
├─────┼────────────────────────────────────────┼──────────────────┼───────────────┤
│ 260 │ Pulp Fiction (1994)                    │          4.16    │           325 │
├─────┼────────────────────────────────────────┼──────────────────┼───────────────┤
│ 316 │ Forrest Gump (1994)               

In [ ]:


# GUI Implementation
def gui_recommender():
    # Widgets for Popularity-based Recommender
    genre_input = widgets.Text(value='', placeholder='Enter genre', description='Genre:')
    min_reviews_input = widgets.IntText(value=100, description='Min Reviews:')
    num_recommendations_pop = widgets.IntText(value=5, description='Num Recommendations:')
    button_pop = widgets.Button(description="Recommend Popular")

    # Widgets for Content-based Recommender
    movie_title_input = widgets.Text(value='', placeholder='Enter movie title', description='Movie Title:')
    num_recommendations_con = widgets.IntText(value=5, description='Num Recommendations:')
    button_con = widgets.Button(description="Recommend Similar")

    # Widgets for Collaborative-based Recommender
    user_id_input = widgets.IntText(value=1, description='User ID:')
    top_k_input = widgets.IntText(value=100, description='Top K Users:')
    num_recommendations_col = widgets.IntText(value=5, description='Num Recommendations:')
    button_col = widgets.Button(description="Recommend Collaborative")

    # Output area
    output = widgets.Output()

    def on_popularity_click(b):
        with output:
            output.clear_output()
            genre = genre_input.value
            min_reviews = min_reviews_input.value
            num_recommendations = num_recommendations_pop.value
            recommendations = popularity_based_recommender(movies, ratings, genre, min_reviews, num_recommendations)
            print("Popularity-based Recommendations:")
            print(tabulate(recommendations, headers=['Title', 'Avg Rating', 'Num Reviews'], tablefmt='fancy_grid'))

    def on_content_click(b):
        with output:
            output.clear_output()
            movie_title = movie_title_input.value
            num_recommendations = num_recommendations_con.value
            recommendations = content_based_recommender(movies, movie_title, num_recommendations)
            if isinstance(recommendations, pd.DataFrame):
                print("Content-based Recommendations:")
                print(tabulate(recommendations, headers=['Title'], tablefmt='fancy_grid'))
            else:
                print(recommendations)

    def on_collaborative_click(b):
        with output:
            output.clear_output()
            target_user_id = user_id_input.value
            top_k = top_k_input.value
            num_recommendations = num_recommendations_col.value
            recommendations = collaborative_recommender(ratings, movies, target_user_id, top_k, num_recommendations)
            if not recommendations.empty:
                print("Collaborative-based Recommendations:")
                print(tabulate(recommendations, headers=['Title', 'Avg Rating'], tablefmt='fancy_grid'))
            else:
                print(f"No recommendations available for User ID {target_user_id}.")

    # Assign button actions
    button_pop.on_click(on_popularity_click)
    button_con.on_click(on_content_click)
    button_col.on_click(on_collaborative_click)

    # Display widgets
    display(
        widgets.VBox([
            widgets.Label("Popularity-based Recommender"),
            genre_input, min_reviews_input, num_recommendations_pop, button_pop,
            widgets.Label("Content-based Recommender"),
            movie_title_input, num_recommendations_con, button_con,
            widgets.Label("Collaborative-based Recommender"),
            user_id_input, top_k_input, num_recommendations_col, button_col,
            output
        ])
    )

# Run the GUI
gui_recommender()
